# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

In [1]:
import  sqlite3

In [2]:
sql  = """
CREATE TABLE ALBUM
(
    ALBUM_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
    TITLE VARCHAR(200)  NOT NULL,
    ARTIST_ID INTEGER  NOT NULL,
    FOREIGN KEY (ARTIST_ID) REFERENCES ARTIST (ARTIST_ID) 
);

CREATE TABLE ARTIST
(
    ARTIST_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    NAME VARCHAR(200),
    COMMENT VARCHAR(400)
);

CREATE TABLE GENRE
(
    GENRE_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    NAME VARCHAR(200)
);

CREATE TABLE PLAY_LIST
(
    PLAY_LIST_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    NAME VARCHAR(200)
);

CREATE TABLE PLAY_LIST_TRACK
(
    PLAY_LIST_ID INTEGER  NOT NULL,
    TRACK_ID INTEGER  NOT NULL,
    CONSTRAINT PLAY_LIST_TRACK PRIMARY KEY  (PLAY_LIST_ID, TRACK_ID),
    FOREIGN KEY (PLAY_LIST_ID) REFERENCES PLAY_LIST (PLAY_LIST_ID),
    FOREIGN KEY (TRACK_ID) REFERENCES TRACK (TRACK_ID) 
);

CREATE TABLE TRACK
(
    TRACK_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    NAME VARCHAR(200)  NOT NULL,
    ALBUM_ID INTEGER,
    MEDIA_TYPE_ID INTEGER  NOT NULL,
    GENRE_ID INTEGER,
    COMPOSER VARCHAR(200),
    MILLISECONDS INTEGER  NOT NULL,
    FOREIGN KEY (ALBUM_ID) REFERENCES ALBUM (ALBUM_ID),
    FOREIGN KEY (GENRE_ID) REFERENCES GENRE (GENRE_ID)
);""" 

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [3]:
con = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = con.cursor()

cur.execute('''
SELECT firstname, lastname
FROM Customer
WHERE country = 'Canada'
''')

res = cur.fetchall()
print(res)

cur.close()
con.close()

[('François', 'Tremblay'), ('Mark', 'Philips'), ('Jennifer', 'Peterson'), ('Robert', 'Brown'), ('Edward', 'Francis'), ('Martha', 'Silk'), ('Aaron', 'Mitchell'), ('Ellie', 'Sullivan')]


2. Найти и вывести на экран названия всех альбомов группы Accept

In [4]:
con = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = con.cursor()

cur.execute('''
SELECT Artist.name, Album.title
FROM Artist, Album
WHERE  Artist.artistid = Album.artistid AND Artist.name = 'Accept'
''')

print(cur.fetchall())
cur.close()
con.close()

[('Accept', 'Balls to the Wall'), ('Accept', 'Restless and Wild')]


3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [6]:
con = sqlite3.connect('PM19-4.sqlite')
cur = con.cursor()

cur.execute('''
CREATE TABLE Student194
(
    ID INTEGER,
    NAME VARCHAR(50)
)
''')

cur.execute('''
INSERT INTO Student194 (ID, NAME)
VALUES (1, 'Dany'),(3, 'Liz')
''')

con.commit()

cur.execute('''
SELECT * FROM Student194
''')
res = cur.fetchall()

cur.close()
cur.close()
con.close()

In [7]:
print(res)

[(1, 'Dany'), (3, 'Liz')]


## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [8]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [9]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

cur.execute('''
CREATE TABLE Recipe(
    ID INTEGER, 
    NAME VARCHAR(200) NOT NULL,
    MINUTES INTEGER  NOT NULL,
    SUBMITTED TEXT,
    DESCRIPTION VARCHAR(200) NOT NULL,
    N_INGREDIENTS INTEGER  NOT NULL
)
''')

con.commit()

cur.close()
con.close()

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [10]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

cur.execute('''
CREATE TABLE Review
(
    ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    USER_ID INTEGER  NOT NULL,
    RECIPE_ID INTEGER  NOT NULL,
    DATE TEXT,
    RATING VARCHAR(300),
    REVIEW VARCHAR(300),
    FOREIGN KEY (RECIPE_ID) REFERENCES RECIPE (ID)
    
)
''')

con.commit()

cur.close()
con.close()

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [11]:
import pandas as pd
import csv

In [12]:
rev = pd.read_csv('reviews_sample.csv') 
to_db_rev = rev.values.tolist()

In [13]:
with open('recipes_sample_with_tags_ingredients.csv','r', encoding="utf8") as f:
    dr_rec = csv.DictReader(f)
    to_db_rec = [(i['id'], i['name'], i['minutes'], i['submitted'], i['description'], i['n_ingredients']) for i in dr_rec]

In [14]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

cur.executemany('''
INSERT INTO Recipe(id, name, minutes, submitted, description, n_ingredients) VALUES (?, ?, ?, ?, ?, ?)''', to_db_rec)

con.commit()

cur.executemany('''
INSERT INTO Review(id, user_id, recipe_id, date, rating, review) VALUES (?, ?, ?, ?, ?, ?)''', to_db_rev)

con.commit()

cur.close()
con.close()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [15]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

cur.execute('''
SELECT name
FROM Recipe
WHERE n_ingredients = 10
''')

res = cur.fetchall()
print(res[:5])

cur.close()
con.close()

[('blepandekager   danish   apple pancakes',), ('kelly s creamy cheddar pea salad',), ('middle eastern   twice baked potatoes',), ('secret ingredient  bbq meatloaf',), ('don t knock it until you try it  beet cake',)]


6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [16]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

cur.execute('''
SELECT name
FROM Recipe
WHERE minutes = (SELECT MAX(minutes) FROM Recipe)
''')

res = cur.fetchall()
print(res[0])

cur.close()
con.close()

('blueberry liqueur',)


7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [20]:
id_ = int(input("Введите id рецепта:"))
rec = pd.read_csv('recipes_sample_with_tags_ingredients.csv', sep = ',', header = 0, parse_dates = [7])
if id_ in rec['id'].tolist():
    con = sqlite3.connect('recipes.db')
    cur = con.cursor()
    
    cur.execute('''
    SELECT *
    FROM Recipe
    WHERE id = ?
    ''', (id_ ,))
    
    res = cur.fetchall()
    print(res)
    
    cur.close()
    con.close()
else:
    print('Такого id не существует')
    

Введите id рецепта:302399
[(302399, 'quick biscuit bread', 20, '2008-05-06', 'this is a wonderful quick bread to make as an acompaniment to most any dish. it is very versatile and delicious 1 :-)', 5)]


8. Найдите кол-во отзывов с рейтингом 5.

In [18]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

cur.execute('''
SELECT COUNT(id)
FROM Review
WHERE rating = 5
''')

res = cur.fetchall()
print(res[0])

cur.close()
con.close()

(91361,)


9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [19]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

cur.execute('''
SELECT COUNT(DISTINCT recipe_id)
FROM Review
WHERE rating < 4
''')

res = cur.fetchall()
print(res[0])

cur.close()
con.close()

(7634,)


10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [21]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

cur.execute('''
SELECT COUNT(submitted)
FROM Recipe
WHERE (minutes >= 15) AND (submitted < '2011-01-01') AND (submitted > '2009-12-31')
''')

res = cur.fetchall()
print(res)

cur.close()
con.close()

[(1319,)]


11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [22]:
#Recipe(id, name, minutes, submitted, description, n_ingredients)
#Review(id, user_id, recipe_id, date, rating, review)

In [23]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

cur.execute('''
SELECT rec.id, rec.name, rev.user_id, rev.date, rev.rating 
FROM Recipe rec, Review rev
WHERE rec.id = rev.recipe_id AND rec.n_ingredients >= 3
ORDER BY recipe_id
''')

res = cur.fetchall()
print(res)

cur.close()
con.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

